### Generate Random lat/lon Event Data and Regrid it

In [1]:
import pandas as pd
import numpy as np
import xarray as xr

# create a Pandas DataFrame
df = pd.DataFrame({
    'date': pd.date_range(start='2023-06-01', periods=1000, freq='D'),
    'latitude': np.random.uniform(-90, 90, 1000),
    'longitude': np.random.uniform(-180, 180, 1000),
    'people': np.random.randint(0, 100, 1000),
    'water': np.random.random(1000),
})

# create bin edges for 1-degree grid
lon_bins = np.arange(-180, 181, 1) # 1-degree grid
lat_bins = np.arange(-90, 91, 1)

# calculate the indices of the bins to which each value in input array belongs
df['lon_bin'] = np.digitize(df['longitude'], lon_bins) - 1
df['lat_bin'] = np.digitize(df['latitude'], lat_bins) - 1

# use bin indices to create bin labels (which are the actual bin values)
df['lon_bin_label'] = lon_bins[df['lon_bin']]
df['lat_bin_label'] = lat_bins[df['lat_bin']]

# group by bins (i.e., group by 1-degree grid cells) and calculate mean in each grid cell
df_grid = df.groupby(['date','lon_bin_label', 'lat_bin_label']).mean().reset_index()

# create grid
lon_grid = lon_bins[:-1]# + 0.5 # grid cell center
lat_grid = lat_bins[:-1]# + 0.5

In [2]:
df_grid.head()

,date,lon_bin_label,lat_bin_label,latitude,longitude,people,water,lon_bin,lat_bin
0,2023-06-01,5,-20,-19.551987,5.188491,25.0,0.939990,185.0,70.0
1,2023-06-02,120,68,68.219120,120.512116,77.0,0.555782,300.0,158.0
2,2023-06-03,12,14,14.616533,12.337145,78.0,0.060956,192.0,104.0
3,2023-06-04,34,73,73.412604,34.413063,49.0,0.805488,214.0,163.0
4,2023-06-05,-145,-10,-9.662638,-144.432208,22.0,0.019650,35.0,80.0


In [3]:
df_grid['latitude'] = df_grid['lat_bin_label']
df_grid['longitude'] = df_grid['lon_bin_label']
df_grid = df_grid[['latitude','longitude','date','people','water']]

In [4]:
df_grid.head()

,latitude,longitude,date,people,water
0,-20,5,2023-06-01,25.0,0.939990
1,68,120,2023-06-02,77.0,0.555782
2,14,12,2023-06-03,78.0,0.060956
3,73,34,2023-06-04,49.0,0.805488
4,-10,-145,2023-06-05,22.0,0.019650


In [5]:
date = np.array(df.date)

In [6]:
# create an empty xarray Dataset to accommodate the 1-degree grid
ds = xr.Dataset(
    coords={
        'date': np.array(date),
        'longitude': lon_grid,
        'latitude': lat_grid
    }
)


In [7]:
# assign 'people' and 'water' DataArrays to the Dataset
ds['people'] = xr.DataArray(
    data=np.full((len(date), len(lon_grid), len(lat_grid)), np.nan), 
    dims=['date','longitude', 'latitude'],
    coords=[date, lon_grid, lat_grid]
)

In [8]:
ds['water'] = xr.DataArray(
    data=np.full((len(date), len(lon_grid), len(lat_grid)), np.nan), 
    dims=['date','longitude', 'latitude'],
    coords=[date, lon_grid, lat_grid]
)

In [9]:
# populate the Dataset with the 'people' and 'water' data
for i, row in df_grid.iterrows():
    ds['people'].loc[dict(date=row['date'], longitude=row['longitude'], latitude=row['latitude'])] = row['people']
    ds['water'].loc[dict(date=row['date'], longitude=row['longitude'], latitude=row['latitude'])] = row['water']

In [10]:
ds

<xarray.Dataset>
Dimensions:    (date: 1000, longitude: 360, latitude: 180)
Coordinates:
  * date       (date) datetime64[ns] 2023-06-01 2023-06-02 ... 2026-02-24
  * longitude  (longitude) int64 -180 -179 -178 -177 -176 ... 176 177 178 179
  * latitude   (latitude) int64 -90 -89 -88 -87 -86 -85 ... 84 85 86 87 88 89
Data variables:
    people     (date, longitude, latitude) float64 nan nan nan ... nan nan nan
    water      (date, longitude, latitude) float64 nan nan nan ... nan nan nan